СМОТР ДАННЫХ

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

КОММЕНТАРИИ

In [2]:
df = pd.read_excel('inputDataComments.xlsx')
df

,commentID,message,masterID,requestID
0,1,Интересная поломка,2,1
1,2,"Очень странно, будем разбираться!",3,2
2,3,Скорее всего потребуется мотор обдува!,2,7
3,4,Интересная поломка,2,1
4,5,"Очень странно, будем разбираться!",3,6


ОБРАЩЕНИЯ

In [3]:
df = pd.read_excel('inputDataRequests.xlsx')
df

,requestID,startDate,homeTechType,homeTechModel,problemDescryption,requestStatus,completionDate,repairParts,masterID,clientID
0,1,2023-06-06,Фен,Ладомир ТА112 белый,Перестал работать,В процессе ремонта,NaN,NaN,2.0,7
1,2,2023-05-05,Тостер,Redmond RT-437 черный,Перестал работать,В процессе ремонта,NaN,NaN,3.0,7
2,3,2022-07-07,Холодильник,Indesit DS 316 W белый,Не морозит одна из камер холодильника,Готова к выдаче,2023-01-01,NaN,2.0,8
3,4,2023-08-02,Стиральная машина,DEXP WM-F610NTMA/WW белый,Перестали работать многие режимы стирки,Новая заявка,NaN,NaN,NaN,8
4,5,2023-08-02,Мультиварка,Redmond RMC-M95 черный,Перестала включаться,Новая заявка,NaN,NaN,NaN,9
5,6,2023-08-02,Фен,Ладомир ТА113 чёрный,Перестал работать,Готова к выдаче,2023-08-03,NaN,2.0,7
6,7,2023-07-09,Холодильник,Indesit DS 314 W серый,"Гудит, но не замораживает",Готова к выдаче,2023-08-03,Мотор обдува морозильной камеры холодильника,2.0,8


ПОЛЬЗОВАТЕЛИ

In [4]:
df = pd.read_excel('inputDataUsers.xlsx')
df

,userID,fio,phone,login,password,type
0,1,Трубин Никита Юрьевич,89210563128,kasoo,root,Менеджер
1,2,Мурашов Андрей Юрьевич,89535078985,murashov123,qwerty,Мастер
2,3,Степанов Андрей Викторович,89210673849,test1,test1,Мастер
3,4,Перина Анастасия Денисовна,89990563748,perinaAD,250519,Оператор
4,5,Мажитова Ксения Сергеевна,89994563847,krutiha1234567,1234567890,Оператор
5,6,Семенова Ясмина Марковна,89994563847,login1,pass1,Мастер
6,7,Баранова Эмилия Марковна,89994563841,login2,pass2,Заказчик
7,8,Егорова Алиса Платоновна,89994563842,login3,pass3,Заказчик
8,9,Титов Максим Иванович,89994563843,login4,pass4,Заказчик
9,10,Иванов Марк Максимович,89994563844,login5,pass5,Мастер


СОЗДАНИЕ БД

In [5]:
import pandas as pd
import sqlite3
import json
from datetime import datetime

def create_combined_table():
    """Создает единую таблицу, объединяющую все данные из XLSX файлов"""
    
    # 1. Загружаем данные из XLSX файлов
    df_requests = pd.read_excel('inputDataRequests.xlsx')
    df_comments = pd.read_excel('inputDataComments.xlsx')
    df_users = pd.read_excel('inputDataUsers.xlsx')
    
    # 2. Объединяем данные
    # Создаем копию запросов
    df_combined = df_requests.copy()
    
    # 3. Обрабатываем комментарии - группируем по requestID
    # Собираем все комментарии для каждого requestID в списки
    comments_grouped = df_comments.groupby('requestID').agg({
        'commentID': list,
        'message': list,
        'masterID': list
    }).reset_index()
    
    comments_grouped = comments_grouped.rename(columns={
        'message': 'comment_messages',
        'masterID': 'comment_masterIDs',
        'commentID': 'comment_ids'
    })
    
    # Объединяем с запросами
    df_combined = df_combined.merge(comments_grouped, left_on='requestID', right_on='requestID', how='left')
    
    # 4. Добавляем информацию о клиенте
    client_info = df_users.set_index('userID')[['fio', 'phone', 'login', 'type']]
    client_info = client_info.rename(columns={
        'fio': 'client_fio',
        'phone': 'client_phone',
        'login': 'client_login',
        'type': 'client_type'
    })
    df_combined = df_combined.merge(client_info, left_on='clientID', right_index=True, how='left')
    
    # 5. Добавляем информацию о мастере (из запросов)
    master_info = df_users.set_index('userID')[['fio', 'phone', 'login', 'type']]
    master_info = master_info.rename(columns={
        'fio': 'master_fio',
        'phone': 'master_phone',
        'login': 'master_login',
        'type': 'master_type'
    })
    df_combined = df_combined.merge(master_info, left_on='masterID', right_index=True, how='left')
    
    # 6. Добавляем информацию о мастерах из комментариев
    # Создаем отдельную таблицу с мастерами комментариев
    comment_masters = pd.DataFrame()
    for idx, row in df_combined.iterrows():
        if isinstance(row.get('comment_masterIDs'), list):
            for master_id in row['comment_masterIDs']:
                comment_masters = pd.concat([comment_masters, pd.DataFrame([{
                    'requestID': row['requestID'],
                    'comment_masterID': master_id
                }])])
    
    if not comment_masters.empty:
        comment_masters = comment_masters.merge(
            master_info.rename(columns={
                'master_fio': 'comment_master_fio',
                'master_phone': 'comment_master_phone',
                'master_login': 'comment_master_login',
                'master_type': 'comment_master_type'
            }),
            left_on='comment_masterID',
            right_index=True,
            how='left'
        )
        
        # Группируем мастеров комментариев по requestID
        comment_masters_grouped = comment_masters.groupby('requestID').agg({
            'comment_masterID': list,
            'comment_master_fio': list,
            'comment_master_phone': list,
            'comment_master_login': list,
            'comment_master_type': list
        }).reset_index()
        
        df_combined = df_combined.merge(comment_masters_grouped, left_on='requestID', right_on='requestID', how='left')
    
    # 7. Переименовываем столбцы для единообразия
    column_mapping = {
        'requestID': 'request_id',
        'startDate': 'start_date',
        'homeTechType': 'tech_type',           # Исправлено
        'homeTechModel': 'tech_model',         # Исправлено
        'problemDescryption': 'problem_description',
        'requestStatus': 'request_status',
        'completionDate': 'completion_date',
        'repairParts': 'repair_parts',
        'masterID': 'master_id',
        'clientID': 'client_id'
    }
    df_combined.rename(columns=column_mapping, inplace=True)
    
    # 8. Преобразуем даты в правильный формат
    date_columns = ['start_date', 'completion_date']
    for col in date_columns:
        if col in df_combined.columns:
            df_combined[col] = pd.to_datetime(df_combined[col], errors='coerce')
    
    # 9. Добавляем вычисляемые столбцы
    df_combined['days_in_process'] = None
    for idx, row in df_combined.iterrows():
        if pd.notnull(row.get('start_date')) and row.get('request_status') != 'Новая заявка':
            if pd.notnull(row.get('completion_date')):
                # Если есть дата завершения
                df_combined.at[idx, 'days_in_process'] = (row['completion_date'] - row['start_date']).days
            else:
                # Если заявка еще в процессе, считаем от текущей даты
                df_combined.at[idx, 'days_in_process'] = (pd.Timestamp.now() - row['start_date']).days
    
    # 10. Добавляем индикатор наличия комментария
    df_combined['has_comment'] = df_combined['comment_messages'].notnull()
    df_combined['comments_count'] = df_combined['comment_messages'].apply(
        lambda x: len(x) if isinstance(x, list) else 0
    )
    
    # 11. Преобразуем списки в строки для лучшей читаемости
    list_columns = ['comment_ids', 'comment_messages', 'comment_masterIDs', 
                    'comment_master_fio', 'comment_master_phone', 
                    'comment_master_login', 'comment_master_type']
    
    for col in list_columns:
        if col in df_combined.columns:
            df_combined[col] = df_combined[col].apply(
                lambda x: '; '.join(map(str, x)) if isinstance(x, list) else ''
            )
    
    # 12. Сортируем и добавляем ID
    df_combined.sort_values(['request_id', 'start_date'], inplace=True)
    df_combined.reset_index(drop=True, inplace=True)
    df_combined['id'] = range(1, len(df_combined) + 1)
    
    # 13. Переупорядочиваем столбцы
    columns_order = [
        'id',
        'request_id',
        'start_date',
        'tech_type',
        'tech_model',
        'problem_description',
        'request_status',
        'completion_date',
        'days_in_process',
        'repair_parts',
        'has_comment',
        'comments_count',
        'comment_ids',
        'comment_messages',
        'master_id',
        'master_fio',
        'master_phone',
        'master_login',
        'master_type',
        'client_id',
        'client_fio',
        'client_phone',
        'client_login',
        'client_type',
        'comment_masterIDs',
        'comment_master_fio',
        'comment_master_phone',
        'comment_master_login',
        'comment_master_type'
    ]
    
    # Выбираем только существующие столбцы
    existing_columns = [col for col in columns_order if col in df_combined.columns]
    
    return df_combined[existing_columns]

def save_all_formats(df_combined):
    """Сохраняет данные во всех форматах"""
    # Excel
    df_combined.to_excel('service_requests_combined.xlsx', index=False)
    print("✓ Excel файл создан: service_requests_combined.xlsx")
    
    # SQLite
    conn = sqlite3.connect('service_requests.db')
    df_combined.to_sql('service_requests', conn, if_exists='replace', index=False)
    
    # Создаем индексы
    cursor = conn.cursor()
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_request_id ON service_requests(request_id)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_client_id ON service_requests(client_id)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_master_id ON service_requests(master_id)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_status ON service_requests(request_status)')
    conn.commit()
    conn.close()
    print("✓ База данных SQLite создана: service_requests.db")
    
    # CSV
    df_combined.to_csv('service_requests_combined.csv', index=False, encoding='utf-8-sig', sep=';')
    print("✓ CSV файл создан: service_requests_combined.csv")
    
    # JSON
    data_dict = df_combined.to_dict(orient='records')
    
    # Преобразуем datetime в строки для JSON
    for record in data_dict:
        for key, value in record.items():
            if isinstance(value, pd.Timestamp):
                record[key] = value.strftime('%Y-%m-%d')
            elif pd.isna(value):
                record[key] = None
    
    with open('service_requests_combined.json', 'w', encoding='utf-8') as f:
        json.dump(data_dict, f, ensure_ascii=False, indent=2)
    print("✓ JSON файл создан: service_requests_combined.json")
    
    # Дополнительные отчеты
    # Сводный отчет по статусам заявок
    status_summary = df_combined.groupby('request_status').agg({
        'request_id': 'count',
        'days_in_process': 'mean'
    }).reset_index()
    status_summary.columns = ['Статус заявки', 'Количество заявок', 'Среднее время обработки (дни)']
    status_summary.to_excel('status_summary.xlsx', index=False)
    print("✓ Сводный отчет по статусам: status_summary.xlsx")
    
    # Отчет по мастерам
    if 'master_fio' in df_combined.columns:
        master_summary = df_combined[df_combined['master_id'].notnull()].groupby(['master_id', 'master_fio']).agg({
            'request_id': 'count',
            'days_in_process': 'mean'
        }).reset_index()
        master_summary.columns = ['ID Мастера', 'ФИО Мастера', 'Количество заявок', 'Среднее время выполнения (дни)']
        master_summary.to_excel('master_summary.xlsx', index=False)
        print("✓ Отчет по мастерам: master_summary.xlsx")
    
    # Отчет по клиентам
    if 'client_fio' in df_combined.columns:
        client_summary = df_combined.groupby(['client_id', 'client_fio']).agg({
            'request_id': 'count'
        }).reset_index()
        client_summary.columns = ['ID Клиента', 'ФИО Клиента', 'Количество заявок']
        client_summary.to_excel('client_summary.xlsx', index=False)
        print("✓ Отчет по клиентам: client_summary.xlsx")

def main():
    """Основная функция"""
    print("Объединение данных из XLSX файлов...")
    print("-" * 50)
    
    try:
        # Создаем объединенную таблицу
        df_combined = create_combined_table()
        
        # Показываем информацию о таблице
        print(f"Создана объединенная таблица с {len(df_combined)} строками")
        print(f"Количество столбцов: {len(df_combined.columns)}")
        print("\nСтруктура таблицы:")
        for col in df_combined.columns:
            print(f"  - {col}")
        
        # Показываем первые строки
        print("\nПервые 3 строки данных:")
        print(df_combined.head(3).to_string(index=False))
        
        # Статистика
        print("\nСтатистика:")
        print(f"Всего заявок: {df_combined['request_id'].nunique()}")
        print(f"Заявок с комментариями: {df_combined['has_comment'].sum()}")
        print(f"Всего комментариев: {df_combined['comments_count'].sum()}")
        
        # Сохраняем во всех форматах
        print("\n" + "-" * 50)
        print("Сохранение данных...")
        save_all_formats(df_combined)
        
        print("\n" + "-" * 50)
        print("ВЫПОЛНЕНО УСПЕШНО!")
        print("\nСозданы файлы:")
        print("1. service_requests_combined.xlsx - Основная таблица в Excel")
        print("2. service_requests.db - База данных SQLite")
        print("3. service_requests_combined.csv - Данные в CSV формате")
        print("4. service_requests_combined.json - Данные в JSON формате")
        print("5. status_summary.xlsx - Сводный отчет по статусам заявок")
        print("6. master_summary.xlsx - Отчет по мастерам")
        print("7. client_summary.xlsx - Отчет по клиентам")
        
        return df_combined
        
    except FileNotFoundError as e:
        print(f"ОШИБКА: Не найден файл {e}")
        print("\nУбедитесь, что все файлы находятся в той же папке:")
        print("1. inputDataRequests.xlsx")
        print("2. inputDataComments.xlsx")
        print("3. inputDataUsers.xlsx")
        return None
    except Exception as e:
        print(f"ОШИБКА: {e}")
        import traceback
        traceback.print_exc()
        return None

if __name__ == "__main__":
    main()

Объединение данных из XLSX файлов...
--------------------------------------------------
Создана объединенная таблица с 7 строками
Количество столбцов: 29

Структура таблицы:
  - id
  - request_id
  - start_date
  - tech_type
  - tech_model
  - problem_description
  - request_status
  - completion_date
  - days_in_process
  - repair_parts
  - has_comment
  - comments_count
  - comment_ids
  - comment_messages
  - master_id
  - master_fio
  - master_phone
  - master_login
  - master_type
  - client_id
  - client_fio
  - client_phone
  - client_login
  - client_type
  - comment_masterIDs
  - comment_master_fio
  - comment_master_phone
  - comment_master_login
  - comment_master_type

Первые 3 строки данных:
 id  request_id start_date   tech_type             tech_model                   problem_description     request_status completion_date days_in_process repair_parts  has_comment  comments_count comment_ids                       comment_messages  master_id                 master_fio  mas

ТАБЛИЦА С ВСЕМИ ДАННЫМИ

In [7]:
import pandas as pd
# Для Jupyter Notebook: импортируем необходимые функции
from IPython.display import display, HTML

df = pd.read_excel('service_requests_combined.xlsx')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Отображаем
display(HTML(df.to_html()))

,id,request_id,start_date,tech_type,tech_model,problem_description,request_status,completion_date,days_in_process,repair_parts,has_comment,comments_count,comment_ids,comment_messages,master_id,master_fio,master_phone,master_login,master_type,client_id,client_fio,client_phone,client_login,client_type,comment_masterIDs,comment_master_fio,comment_master_phone,comment_master_login,comment_master_type
0,1,1,2023-06-06,Фен,Ладомир ТА112 белый,Перестал работать,В процессе ремонта,NaT,930.0,NaN,True,2,1; 4,Интересная поломка; Интересная поломка,2.0,Мурашов Андрей Юрьевич,8.953508e+10,murashov123,Мастер,7,Баранова Эмилия Марковна,89994563841,login2,Заказчик,2; 2,Мурашов Андрей Юрьевич; Мурашов Андрей Юрьевич,89535078985; 89535078985,murashov123; murashov123,Мастер; Мастер
1,2,2,2023-05-05,Тостер,Redmond RT-437 черный,Перестал работать,В процессе ремонта,NaT,962.0,NaN,True,1,2,"Очень странно, будем разбираться!",3.0,Степанов Андрей Викторович,8.921067e+10,test1,Мастер,7,Баранова Эмилия Марковна,89994563841,login2,Заказчик,3,Степанов Андрей Викторович,89210673849,test1,Мастер
2,3,3,2022-07-07,Холодильник,Indesit DS 316 W белый,Не морозит одна из камер холодильника,Готова к выдаче,2023-01-01,178.0,NaN,False,0,NaN,NaN,2.0,Мурашов Андрей Юрьевич,8.953508e+10,murashov123,Мастер,8,Егорова Алиса Платоновна,89994563842,login3,Заказчик,NaN,NaN,NaN,NaN,NaN
3,4,4,2023-08-02,Стиральная машина,DEXP WM-F610NTMA/WW белый,Перестали работать многие режимы стирки,Новая заявка,NaT,NaN,NaN,False,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,Егорова Алиса Платоновна,89994563842,login3,Заказчик,NaN,NaN,NaN,NaN,NaN
4,5,5,2023-08-02,Мультиварка,Redmond RMC-M95 черный,Перестала включаться,Новая заявка,NaT,NaN,NaN,False,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,Титов Максим Иванович,89994563843,login4,Заказчик,NaN,NaN,NaN,NaN,NaN
5,6,6,2023-08-02,Фен,Ладомир ТА113 чёрный,Перестал работать,Готова к выдаче,2023-08-03,1.0,NaN,True,1,5,"Очень странно, будем разбираться!",2.0,Мурашов Андрей Юрьевич,8.953508e+10,murashov123,Мастер,7,Баранова Эмилия Марковна,89994563841,login2,Заказчик,3,Степанов Андрей Викторович,89210673849,test1,Мастер
6,7,7,2023-07-09,Холодильник,Indesit DS 314 W серый,"Гудит, но не замораживает",Готова к выдаче,2023-08-03,25.0,Мотор обдува морозильной камеры холодильника,True,1,3,Скорее всего потребуется мотор обдува!,2.0,Мурашов Андрей Юрьевич,8.953508e+10,murashov123,Мастер,8,Егорова Алиса Платоновна,89994563842,login3,Заказчик,2,Мурашов Андрей Юрьевич,89535078985,murashov123,Мастер


ДИАГРАММЫ

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
from datetime import datetime

# Настройка стиля для диаграмм
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

def create_service_analytics_pdf():
    """
    Создает PDF файл с диаграммами анализа сервисных заявок
    """
    # Загружаем данные
    print("Загрузка данных...")
    
    # Основные данные заявок
    requests_df = pd.read_excel('inputDataRequests.xlsx')
    
    # Комментарии
    comments_df = pd.read_excel('inputDataComments.xlsx')
    
    # Пользователи
    users_df = pd.read_excel('inputDataUsers.xlsx')
    
    # Объединенные данные (если нужны дополнительные метрики)
    combined_df = pd.read_excel('service_requests_combined.xlsx')
    
    # Создаем PDF файл
    pdf_filename = 'service_requests_analysis_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.pdf'
    
    with PdfPages(pdf_filename) as pdf:
        print(f"Создание диаграмм в PDF: {pdf_filename}")
        
        # ========== ДИАГРАММА 1: Распределение заявок по статусам ==========
        plt.figure(figsize=(10, 6))
        
        # Группируем по статусам
        status_counts = requests_df['requestStatus'].value_counts()
        
        # Создаем круговую диаграмму
        colors = ['#4CAF50', '#FF9800', '#2196F3', '#F44336']
        plt.pie(status_counts.values, labels=status_counts.index, autopct='%1.1f%%',
                colors=colors[:len(status_counts)], startangle=90, shadow=True)
        plt.title('Распределение заявок по статусам', fontsize=16, fontweight='bold')
        plt.axis('equal')
        
        pdf.savefig()  # Сохраняем в PDF
        plt.close()
        
        # ========== ДИАГРАММА 2: Количество заявок по типам техники ==========
        plt.figure(figsize=(12, 7))
        
        tech_counts = requests_df['homeTechType'].value_counts().head(10)
        
        bars = plt.bar(tech_counts.index, tech_counts.values, color='#2196F3', edgecolor='black')
        
        # Добавляем значения на столбцы
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                    f'{int(height)}', ha='center', va='bottom')
        
        plt.title('Количество заявок по типам техники', fontsize=16, fontweight='bold')
        plt.xlabel('Тип техники', fontsize=12)
        plt.ylabel('Количество заявок', fontsize=12)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        
        pdf.savefig()
        plt.close()
        
        # ========== ДИАГРАММА 3: Распределение заявок по месяцам ==========
        plt.figure(figsize=(12, 7))
        
        # Преобразуем даты
        requests_df['startDate'] = pd.to_datetime(requests_df['startDate'])
        requests_df['month'] = requests_df['startDate'].dt.to_period('M')
        
        # Группируем по месяцам
        monthly_counts = requests_df.groupby('month').size()
        
        # Преобразуем периоды в строки для отображения
        monthly_index = monthly_counts.index.astype(str)
        
        plt.plot(monthly_index, monthly_counts.values, marker='o', linewidth=2, markersize=8,
                color='#FF5722', markerfacecolor='white', markeredgewidth=2)
        
        # Добавляем точки с значениями
        for x, y in zip(range(len(monthly_index)), monthly_counts.values):
            plt.text(x, y + 0.1, f'{y}', ha='center', va='bottom')
        
        plt.title('Динамика заявок по месяцам', fontsize=16, fontweight='bold')
        plt.xlabel('Месяц', fontsize=12)
        plt.ylabel('Количество заявок', fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.xticks(rotation=45)
        plt.tight_layout()
        
        pdf.savefig()
        plt.close()
        
        # ========== ДИАГРАММА 4: Распределение заявок по мастерам ==========
        plt.figure(figsize=(12, 7))
        
        # Объединяем с данными мастеров
        master_requests = pd.merge(requests_df, users_df[['userID', 'fio', 'type']], 
                                 left_on='masterID', right_on='userID', how='left')
        
        # Фильтруем только мастеров
        master_requests = master_requests[master_requests['type'] == 'Мастер']
        
        if not master_requests.empty:
            # Берем только фамилии для краткости
            master_requests['short_name'] = master_requests['fio'].apply(lambda x: x.split()[0])
            master_counts = master_requests['short_name'].value_counts()
            
            colors_master = plt.cm.Set3(range(len(master_counts)))
            plt.barh(master_counts.index, master_counts.values, color=colors_master, edgecolor='black')
            
            plt.title('Распределение заявок по мастерам', fontsize=16, fontweight='bold')
            plt.xlabel('Количество заявок', fontsize=12)
            plt.ylabel('Мастер', fontsize=12)
            
            # Добавляем значения
            for i, v in enumerate(master_counts.values):
                plt.text(v + 0.1, i, f'{v}', va='center')
                
            plt.tight_layout()
        else:
            plt.text(0.5, 0.5, 'Нет данных о назначенных мастерах', 
                    ha='center', va='center', fontsize=14)
        
        pdf.savefig()
        plt.close()
        
        # ========== ДИАГРАММА 5: Статистика комментариев ==========
        plt.figure(figsize=(10, 6))
        
        # Группируем комментарии по requestID
        comments_per_request = comments_df.groupby('requestID').size()
        
        # Считаем распределение
        comments_distribution = comments_per_request.value_counts().sort_index()
        
        bars = plt.bar(comments_distribution.index.astype(str), 
                      comments_distribution.values, color='#9C27B0', edgecolor='black')
        
        # Добавляем значения
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                    f'{int(height)}', ha='center', va='bottom')
        
        plt.title('Распределение количества комментариев на заявку', fontsize=16, fontweight='bold')
        plt.xlabel('Количество комментариев', fontsize=12)
        plt.ylabel('Количество заявок', fontsize=12)
        plt.tight_layout()
        
        pdf.savefig()
        plt.close()
        
        # ========== ДИАГРАММА 6: Соотношение типов пользователей ==========
        plt.figure(figsize=(10, 6))
        
        user_type_counts = users_df['type'].value_counts()
        
        # Используем взрыв для выделения секторов
        explode = [0.1 if i == 0 else 0 for i in range(len(user_type_counts))]
        
        plt.pie(user_type_counts.values, labels=user_type_counts.index, 
                autopct='%1.1f%%', explode=explode, shadow=True,
                colors=['#FFB74D', '#81C784', '#64B5F6', '#BA68C8'])
        
        plt.title('Распределение типов пользователей', fontsize=16, fontweight='bold')
        plt.axis('equal')
        
        pdf.savefig()
        plt.close()
        
        # ========== ДИАГРАММА 7: Статусы заявок по типам техники ==========
        plt.figure(figsize=(14, 8))
        
        # Создаем сводную таблицу
        pivot_data = pd.crosstab(requests_df['homeTechType'], requests_df['requestStatus'])
        
        # Берем топ-5 типов техники
        top_tech = requests_df['homeTechType'].value_counts().head(5).index
        pivot_data = pivot_data.loc[top_tech]
        
        ax = pivot_data.plot(kind='bar', stacked=True, figsize=(14, 8),
                           color=['#4CAF50', '#FF9800', '#2196F3', '#F44336'][:len(pivot_data.columns)])
        
        plt.title('Статусы заявок по типам техники (Топ-5)', fontsize=16, fontweight='bold')
        plt.xlabel('Тип техники', fontsize=12)
        plt.ylabel('Количество заявок', fontsize=12)
        plt.legend(title='Статус заявки', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.xticks(rotation=45, ha='right')
        
        # Добавляем значения на столбцы
        for container in ax.containers:
            ax.bar_label(container, fmt='%d', label_type='center', fontsize=10)
        
        plt.tight_layout()
        
        pdf.savefig()
        plt.close()
        
        # ========== СТРАНИЦА С СВОДНОЙ СТАТИСТИКОЙ ==========
        fig, ax = plt.subplots(figsize=(11, 8))
        ax.axis('tight')
        ax.axis('off')
        
        # Собираем статистику
        stats_text = [
            "СТАТИСТИКА СЕРВИСНЫХ ЗАЯВОК",
            "=" * 40,
            f"Всего заявок: {len(requests_df)}",
            f"Заявок в работе: {len(requests_df[requests_df['requestStatus'] == 'В процессе ремонта'])}",
            f"Завершенных заявок: {len(requests_df[requests_df['requestStatus'] == 'Готова к выдаче'])}",
            f"Новых заявок: {len(requests_df[requests_df['requestStatus'] == 'Новая заявка'])}",
            "",
            f"Всего комментариев: {len(comments_df)}",
            f"Заявок с комментариями: {comments_df['requestID'].nunique()}",
            f"Уникальных мастеров: {users_df[users_df['type'] == 'Мастер']['userID'].nunique()}",
            f"Уникальных клиентов: {users_df[users_df['type'] == 'Заказчик']['userID'].nunique()}",
            "",
            "ПОПУЛЯРНЫЕ ТИПЫ ТЕХНИКИ:"
        ]
        
        # Добавляем топ-3 типа техники
        for i, (tech, count) in enumerate(tech_counts.head(3).items(), 1):
            stats_text.append(f"  {i}. {tech}: {count} заявок")
        
        stats_text.extend([
            "",
            "АКТИВНЫЕ МАСТЕРА:"
        ])
        
        # Добавляем топ-3 мастеров
        if not master_requests.empty:
            for i, (master, count) in enumerate(master_counts.head(3).items(), 1):
                stats_text.append(f"  {i}. {master}: {count} заявок")
        
        stats_text.extend([
            "",
            f"Дата создания отчета: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
        ])
        
        # Отображаем текст
        plt.text(0.1, 0.95, '\n'.join(stats_text), 
                fontsize=12, fontfamily='monospace',
                verticalalignment='top',
                transform=ax.transAxes)
        
        pdf.savefig()
        plt.close()
        
        print(f"PDF файл успешно создан: {pdf_filename}")
    
    return pdf_filename

# Основная функция
if __name__ == "__main__":
    try:
        # Создаем PDF с диаграммами
        pdf_file = create_service_analytics_pdf()
        
        print("\n" + "="*60)
        print("ОТЧЕТ УСПЕШНО СОЗДАН!")
        print("="*60)
        print(f"Файл: {pdf_file}")
        print("\nСодержание отчета:")
        print("1. Распределение заявок по статусам")
        print("2. Количество заявок по типам техники")
        print("3. Динамика заявок по месяцам")
        print("4. Распределение заявок по мастерам")
        print("5. Статистика комментариев")
        print("6. Распределение типов пользователей")
        print("7. Статусы заявок по типам техники")
        print("8. Сводная статистика")
        
    except FileNotFoundError as e:
        print(f"Ошибка: Файл не найден - {e}")
        print("Убедитесь, что все файлы находятся в той же директории:")
        print("  - inputDataRequests.xlsx")
        print("  - inputDataComments.xlsx")
        print("  - inputDataUsers.xlsx")
        print("  - service_requests_combined.xlsx")
    except Exception as e:
        print(f"Произошла ошибка: {e}")

Загрузка данных...
Создание диаграмм в PDF: service_requests_analysis_20251222_152755.pdf
PDF файл успешно создан: service_requests_analysis_20251222_152755.pdf

ОТЧЕТ УСПЕШНО СОЗДАН!
Файл: service_requests_analysis_20251222_152755.pdf

Содержание отчета:
1. Распределение заявок по статусам
2. Количество заявок по типам техники
3. Динамика заявок по месяцам
4. Распределение заявок по мастерам
5. Статистика комментариев
6. Распределение типов пользователей
7. Статусы заявок по типам техники
8. Сводная статистика


<Figure size 1400x800 with 0 Axes>